# Training a Simmple LUTNN

In [5]:
import torch
import torch.optim as optim
from lutnn.lutlayer import LUTLayer, Aggregation

In [6]:
from utils.mnist import load_mnist_dataset
train_loader, test_loader, input_dim_dataset, num_classes = load_mnist_dataset(batch_size=128)

In [7]:
# Define a simple LUTNN
class SimpleLUTNN(torch.nn.Module):
    def __init__(self):
        super(SimpleLUTNN, self).__init__()
        self.layer1 = LUTLayer(input_dim=input_dim_dataset, lut_size=6, n_luts=2048)
        self.layer2 = LUTLayer(input_dim=2048, lut_size=6, n_luts=4000)
        self.layer3 = Aggregation(num_classes=num_classes, tau = 10.)

    def forward(self, x):
        x = x.view(-1, 28*28)  # Flatten the input
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        return x

In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
model = SimpleLUTNN().to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

cpu


In [9]:
# Training loop
num_epochs = 2
for epoch in range(num_epochs):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        
        # Forward pass
        outputs = model(data)
        loss = criterion(outputs, target)
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch_idx % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{batch_idx}/{len(train_loader)}], Loss: {loss.item():.4f}')

# Evaluation
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        outputs = model(data)
        _, predicted = torch.max(outputs.data, 1)
        total += target.size(0)
        correct += (predicted == target).sum().item()

print(f'Test Accuracy: {100 * correct / total:.2f}%')

Epoch [1/2], Step [0/468], Loss: 2.3092
Epoch [1/2], Step [100/468], Loss: 0.7295
Epoch [1/2], Step [200/468], Loss: 0.4540
Epoch [1/2], Step [300/468], Loss: 0.3075
Epoch [1/2], Step [400/468], Loss: 0.3427
Epoch [2/2], Step [0/468], Loss: 0.2399
Epoch [2/2], Step [100/468], Loss: 0.1354
Epoch [2/2], Step [200/468], Loss: 0.2763
Epoch [2/2], Step [300/468], Loss: 0.3786
Epoch [2/2], Step [400/468], Loss: 0.2295
Test Accuracy: 91.31%
